## Binary Classifier
This notebook builds a binary classification model and performs performance and fairness assessments for it.

The model predicts whether a loan applicant is unqualified or qualified based on their income, credit, etc.

Dataset preparation notebook available [here](https://github.com/credo-ai/customer_demos/blob/prod/prod/d3_loan_approval/data_preparation.ipynb).

In [1]:
#Loading Libraries
import warnings
warnings.filterwarnings('ignore')
import pandas as pd

from sklearn.linear_model import LogisticRegression

from sklearn.model_selection import train_test_split
import pickle

In [2]:
sensitive_feature_keys = ['Gender', 'Race']
label_key = 'loan_approved'

In [3]:
df = pd.read_pickle('../frozen_data/binary/loan_processed.pkl')

In [4]:
sf = df[sensitive_feature_keys]
target = df[label_key]
features = df.drop(sensitive_feature_keys + [label_key], axis=1)


In [5]:
X_train, X_test, y_train, y_test, sf_train, sf_test = train_test_split(
    features, target, sf, random_state=0, test_size=0.3
    )

In [6]:
clf = LogisticRegression(random_state=0)
clf.fit(X_train, y_train)

LogisticRegression(random_state=0)

In [7]:
df_training = pd.concat([sf_train, X_train, y_train], axis=1)
df_validation = pd.concat([sf_test, X_test, y_test], axis=1)

In [8]:
from credoai.lens import Lens
from credoai.artifacts import TabularData, ClassificationModel
from credoai.evaluators import *
from credoai.governance import Governance
import numpy as np

In [9]:
credo_model = ClassificationModel(
    'loan_default_classifier',
    clf
)
train_data = TabularData(
    name='blarg',
    X=X_train,
    y=y_train,
    sensitive_features=sf_train
)
test_data = TabularData(
    name='loan_val',
    X=X_test,
    y=y_test,
    sensitive_features=sf_test
)

In [10]:
validation_data_dict = {
    'name': 'loan_val', 
    'val_features': X_test, 
    'val_labels': y_test, 
    'label_name': 'loan_approved', 
    'sensitive_features': sf_test
}

with open('../frozen_data/binary/loan_validation.pkl', 'wb') as f:
    pickle.dump(validation_data_dict, f)

train_data_dict = {
    'name': 'loan_train',
    'train_features': X_train,
    'train_labels': y_train,
    'label_name': 'loan_approved',
    'sensitive_features': sf_train
}

with open('../frozen_data/binary/loan_train.pkl', 'wb') as f:
    pickle.dump(train_data_dict, f)

In [11]:
# pipeline scan be specifed using a sklearn-like style
metrics = ["false_negative_rate", "average_precision_score"]
pipeline = [
    (Performance(metrics), 'Performance'),
    (ModelFairness(metrics), 'ModelFairness'),
]

pipeline_info = {'metrics': metrics, 'assessments': ['Performance', 'ModelFairness']}

with open('../frozen_results/binary/pipeline_info.pkl', 'wb') as f:
    pickle.dump(pipeline_info, f)


In [12]:
lens = Lens(
    model=credo_model,
    # training_data=train_data,
    assessment_data=test_data,
    pipeline=pipeline
)

# lens.add(ModelFairness(metrics), "ModelFairness")

lens.run()

2022-10-24 15:26:51,761 - lens - INFO - Evaluator Performance added to pipeline. 
2022-10-24 15:26:51,766 - lens - INFO - Evaluator ModelFairness added to pipeline. Dataset used: assessment_data. Sensitive feature: Gender
2022-10-24 15:26:51,771 - lens - INFO - Evaluator ModelFairness added to pipeline. Dataset used: assessment_data. Sensitive feature: Race
2022-10-24 15:26:51,771 - lens - INFO - Running evaluation for step: Performance
2022-10-24 15:26:51,773 - lens - INFO - Running evaluation for step: ModelFairness


In [13]:
lens.get_results()

{'Performance': [                      type     value
  0      false_negative_rate  0.034351
  1  average_precision_score  0.859587],
 'ModelFairness': [                             type     value
  0      false_negative_rate_parity  0.006958
  1  average_precision_score_parity  0.066035,
      Race                     type     value
  0  Black      false_negative_rate  0.039216
  1   NHPI      false_negative_rate  0.032258
  2  White      false_negative_rate  0.033333
  3  Black  average_precision_score  0.860209
  4   NHPI  average_precision_score  0.803702
  5  White  average_precision_score  0.869738]}

In [14]:
results = lens.get_results()
for assessment, result in results.items():
    with open('../frozen_results/binary/binary_clf_' + assessment + '_results.pkl', 'wb') as f:
        pickle.dump(result, f)

In [15]:
for assessment, result in results.items():
    print(result)

[                      type     value
0      false_negative_rate  0.034351
1  average_precision_score  0.859587]
[                             type     value
0      false_negative_rate_parity  0.006958
1  average_precision_score_parity  0.066035,     Race                     type     value
0  Black      false_negative_rate  0.039216
1   NHPI      false_negative_rate  0.032258
2  White      false_negative_rate  0.033333
3  Black  average_precision_score  0.860209
4   NHPI  average_precision_score  0.803702
5  White  average_precision_score  0.869738]


In [16]:
results['ModelFairness']

[                             type     value
 0      false_negative_rate_parity  0.006958
 1  average_precision_score_parity  0.066035,
     Race                     type     value
 0  Black      false_negative_rate  0.039216
 1   NHPI      false_negative_rate  0.032258
 2  White      false_negative_rate  0.033333
 3  Black  average_precision_score  0.860209
 4   NHPI  average_precision_score  0.803702
 5  White  average_precision_score  0.869738]